In [1]:
import json
from glob import glob

In [8]:
# CONFIG = 'hopper_custom_rl' 
# DATASET = 'H-MIXED-RT-4'
# model_dir_glob = "/rds/project/rds-eWkDxBhxBrQ/dimorl/code/mopo/ray_mopo/Hopper_H-MIXED-RT-4/*/*"
# rollout_lengths = {1, 5, 50}

# CONFIG = 'halfcheetah_custom_rl' 
# DATASET = 'MIXED-RT-1'
# model_dir_glob = "/rds/project/rds-eWkDxBhxBrQ/dimorl/code/mopo/ray_mopo/HalfCheetah_MIXED-RT-1/*/*"
# rollout_lengths = {5, 10} #{1, 5, 20}
# mopo_penalty_coeffs = {0, 5}

# CONFIG = 'halfcheetah_custom_rl' 
# DATASET = 'MIXED-RT-4'
# model_dir_glob = "/rds/project/rds-eWkDxBhxBrQ/dimorl/code/mopo/ray_mopo/HalfCheetah_MIXED-RT-4/*/*"
# rollout_lengths = {10, } #{5, 10} #{1, 5, 20}
# mopo_penalty_coeffs = {5,} #{0, 5}

# CONFIG = 'walker2d_custom_rl' 
# DATASET = 'W-MIXED-RT-1' 
# model_dir_glob = "/rds/project/rds-eWkDxBhxBrQ/dimorl/code/mopo/ray_mopo/Walker2d_W-MIXED-RT-1/*/*"
# rollout_lengths = {5, 10}
# mopo_penalty_coeffs = {0, } #{0, 1}

# CONFIG = 'walker2d_custom_rl' 
# DATASET = 'W-MIXED-RT-4' 
# model_dir_glob = "/rds/project/rds-eWkDxBhxBrQ/dimorl/code/mopo/ray_mopo/Walker2d_W-MIXED-RT-4/*/*"
# rollout_lengths = {5, 10}
# mopo_penalty_coeffs = {0, } #{0, 1}

# CONFIG = 'walker2d_custom_rl' 
# DATASET = 'W-MIXED-RT-5' 
# model_dir_glob = "/rds/project/rds-eWkDxBhxBrQ/dimorl/code/mopo/ray_mopo/Walker2d_W-MIXED-RT-5/*/*"
# rollout_lengths = {5, 10}
# mopo_penalty_coeffs = {0, } #{0, 1}

CONFIG = 'humonoid_custom_rl' 
DATASET = 'Humanoid-Novice-10' 
model_dir_glob = "/rds/project/rds-eWkDxBhxBrQ/dimorl/code/mopo/ray_mopo/Humanoid_Humanoid-Novice-10/*/*"
rollout_lengths = {5, 10, 20}
mopo_penalty_coeffs = {0, 1, 5} #{0, 1}

# mopo_penalty_coeffs = {0, 1, 5}
seeds = {1, 2, 3}
# seeds = {1,}

In [9]:
dir_list = glob(model_dir_glob, recursive = True)

def filter_dir(x):
    last = x.split('/')[-1]
    return last.startswith('s') 

with_s = list(filter(filter_dir, dir_list))
with_s

['/rds/project/rds-eWkDxBhxBrQ/dimorl/code/mopo/ray_mopo/Humanoid_Humanoid-Novice-10/rexb20_br_rexF/s5_2023-04-11_16-58-19_d_kjei1',
 '/rds/project/rds-eWkDxBhxBrQ/dimorl/code/mopo/ray_mopo/Humanoid_Humanoid-Novice-10/rexb10_br_rexF/s5_2023-04-11_16-58-19cws934iq',
 '/rds/project/rds-eWkDxBhxBrQ/dimorl/code/mopo/ray_mopo/Humanoid_Humanoid-Novice-10/rexb5_br_rexF/s5_2023-04-11_16-58-19t6agz8hb']

In [10]:
dir_list = glob(model_dir_glob, recursive = True)

def filter_dir(x):
    last = x.split('/')[-1]
    return last.startswith('s') 

with_s = list(filter(filter_dir, dir_list))
with_s = ["/".join(model_dir.split('/')[-3:])for model_dir in with_s]
model_load_dirs = with_s
with_s

['Humanoid_Humanoid-Novice-10/rexb20_br_rexF/s5_2023-04-11_16-58-19_d_kjei1',
 'Humanoid_Humanoid-Novice-10/rexb10_br_rexT/s10_2023-04-11_22-34-16vp8koidb',
 'Humanoid_Humanoid-Novice-10/rexb10_br_rexF/s5_2023-04-11_16-58-19cws934iq',
 'Humanoid_Humanoid-Novice-10/rexb5_br_rexF/s5_2023-04-11_16-58-19t6agz8hb']

In [13]:
model_load_dirs = [mld for mld in  model_load_dirs if 'rexb10_br_rexT' not in mld]
# model_load_dirs = [mld for mld in  model_load_dirs if '/s6_' in mld and ('rp5_' in mld or 'rp_' in mld) and 'std' not in mld]
# model_load_dirs

In [14]:
model_load_dirs

['Humanoid_Humanoid-Novice-10/rexb20_br_rexF/s5_2023-04-11_16-58-19_d_kjei1',
 'Humanoid_Humanoid-Novice-10/rexb10_br_rexF/s5_2023-04-11_16-58-19cws934iq',
 'Humanoid_Humanoid-Novice-10/rexb5_br_rexF/s5_2023-04-11_16-58-19t6agz8hb']

In [15]:
# model_load_dirs = ['HalfCheetah_MIXED-RT-5/default/s1_2023-03-06_11-13-45xh8hsuja']

In [18]:
list_to_dump = []

for mld in model_load_dirs:
    bnn_name = mld.split('/')[-2]
    bnn_seed = mld.split('/')[-1].split('_')[0]
    for rl in rollout_lengths:
        for mp in mopo_penalty_coeffs:
            for seed in seeds:
                exp_name = f'{bnn_name}{bnn_seed}_mp{mp}rl{rl}'
                list_to_dump.append(dict(config=CONFIG, dataset=DATASET, rollout_length=rl, mopo_penalty_coeff=mp,
                                    seed=seed, model_load_dir=mld, exp_name=exp_name)) 

with open(f"{DATASET}_rex_exp_params.json", "w") as outfile:
    json.dump(list_to_dump, outfile, indent="")

In [19]:
f"{DATASET}_rex_exp_params.json"

'Humanoid-Novice-10_rex_exp_params.json'